In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [ ]:
from math import radians, cos, sin, asin, sqrt

def dist(a, b):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    x1, y1, x2, y2 = map(radians, [a[0], a[1], b[0], b[1]])
    dx = x2 - x1
    dy = y2 - y1
    a = sin(dy/2)**2 + cos(y1) * cos(y2) * sin(dx/2)**2
    c = 2 * asin(sqrt(a))
    # Earth`s radius in kilometers
    km = 6372.795 * c
    return km
def f(a):
    return a+2

In [ ]:
meteo_path = "G:/1_Data1/90_BaikalJun2022Sea/Meteo/"
origin_path = "G:/1_Data1/90_BaikalJun2022Sea/jun2022sea.dat"

In [ ]:
origin_df = pd.read_csv(origin_path,
                        header=0,
                        skiprows=1,
                        sep="\t",
                        na_values='--',
                        decimal=',',
                        parse_dates=['DateTime'],
                        dayfirst=True,
                        )
#origin_df.info()
origin_df.sort_values(by='DateTime', inplace=True)

In [ ]:
print(origin_df.columns)

In [ ]:
cols = ['Longitude', 'Latitude', 'TempAir', 'PressAir',
       'TempEqu1', 'dateSec', 'vCO2Air', 'vCH4Air', 'pCO2Air', 'pCH4Air', 'cCO2Wtr', 'pCO2Wtr', 'cCH4Wtr', 'pCH4Wtr']
cols_dict = {}
for item in cols:
    cols_dict[item] = 'mean'

In [ ]:
origin_df_minutly = origin_df.resample('1T', on='DateTime').agg(cols_dict).ffill()
origin_df_minutly

In [ ]:
origin_df['lat_shifted'] = origin_df['Latitude'].shift(1)
origin_df['long_shifted'] = origin_df['Longitude'].shift(1)
origin_df['distance'] = 0
origin_df['vessel_speed'] = 0

In [ ]:
meteo_files_list = glob.glob(meteo_path+"*.txt")
dfl = []
for f in meteo_files_list:
    df = pd.read_csv(f,
                     encoding_errors="ignore",
                     header=None,
                     skiprows=1,
                     sep=" ",
                     names=['date', 'time', 't_air', 'w_hor', 'w_dir', 'wx', 'wy', 'wz', 'press', 'rh']
                    )
    dfl.append(df)

meteo_df = pd.concat(dfl)
meteo_df['DateTime'] = pd.to_datetime(meteo_df['date']+' '+meteo_df['time'], dayfirst=True, cache=True)
meteo_df.sort_values(by='DateTime', inplace=True)
meteo_df


In [ ]:
print(df.columns)
cols = ['t_air', 'w_hor', 'w_dir', 'press', 'rh']
cols_dict = {}
for item in cols:
    cols_dict[item] = 'mean'

In [ ]:
meteo_df_minutly = meteo_df.resample("1T", on="DateTime", ).agg(cols_dict).ffill()
meteo_df_minutly

In [ ]:
df = pd.merge(origin_df_minutly, meteo_df_minutly, how='outer', on='DateTime')
df.sort_values(by='DateTime', inplace=True)
df

In [ ]:
dLong = np.radians(df['Longitude'] - df['Longitude'].shift(1))
dLat = np.radians(df['Latitude'] - df['Latitude'].shift(1))
a = np.sin(dLat/2)**2 + np.cos(df['Longitude']) * np.cos(df['Longitude'].shift(1)) * np.sin(dLong)**2
df['speed'] = (2 * np.arcsin(np.sqrt(a)) * 6372.8) * 60

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16,5)
ax.set_ylim(0, 80)
ax.set_xlim(pd.to_datetime('2022/06/01'),pd.to_datetime('2022/06/15'))
ax.grid()
ax.plot(df.index, 'speed', 'g-', data=df)

In [ ]:
df.to_csv('G:/1_Data1/90_BaikalJun2022Sea/jun2022sea_pandas.out', sep='\t')

In [ ]:
plt.isinteractive()